### Importing Necessary Libraries

In [1]:
import numpy as np, glob, os, shutil, matplotlib.pyplot as plt, cv2 as cv, random, tifffile as tiff, imagecodecs
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,ReLU,concatenate,Activation, Dropout,Conv2DTranspose, Concatenate, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.metrics import categorical_crossentropy

### Model Architecture

In [64]:
mask_dir = 'D:/LCDC/Dataset/Training/mask/mask_CP/'
wsi_dir = 'D:/LCDC/Dataset/Training/wsi/WSI_CP/'

In [3]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x
  
def get_unet(input, n_filters = 16, dropout = 0.1, batchnorm = True):
    # Contracting Path
    c1 = conv2d_block(input, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = conv2d_block(p1, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = conv2d_block(p2, n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 32, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters = n_filters * 64, kernel_size = 3, batchnorm = batchnorm)
    
    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 32, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 32, kernel_size = 3, batchnorm = batchnorm)
    
    u7 = Conv2DTranspose(n_filters * 16, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    
    u8 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    
    u9 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input], outputs=[outputs])
    return model

### Using ImageDataGenerator

In [14]:
def data_gen(img_path, mask_path):
    seed = 909
    wsi_datagen = ImageDataGenerator(rescale=1/255)
    mask_datagen = ImageDataGenerator(rescale=1/255)

    wsi_gen = wsi_datagen.flow_from_directory(img_path, batch_size = 64, class_mode = None, seed = seed)
    mask_gen = mask_datagen.flow_from_directory(mask_path, batch_size = 64, class_mode = None, seed = seed, color_mode='grayscale')

    return wsi_gen, mask_gen

In [15]:
train_imgs, train_masks = data_gen(wsi_dir, mask_dir)

Found 1087423 images belonging to 1 classes.
Found 1087423 images belonging to 1 classes.


In [26]:
img_batch, mask_batch = next(train_imgs), next(train_masks)
print(img_batch.shape, ' ### ', mask_batch.shape)

(64, 256, 256, 3)  ###  (64, 256, 256, 1)


### Using tf.data.Dataset

In [65]:
img_train_batch = tf.data.Dataset.list_files(wsi_dir+'*')
mask_train_batch = tf.data.Dataset.list_files(mask_dir+'*')

In [ ]:
list(img_train_batch.as_numpy_iterator())

### Brute force logic

In [102]:
def make_batches(wsi_dir, mask_dir, batch_size = 64):
    train_img_batches, train_mask_batches,img_batch, mask_batch,counter = [],[],[],[],0
    for img, mask in zip(glob.glob(wsi_dir+'*'), glob.glob(mask_dir+'*')):
        if counter == batch_size:
            train_img_batches.append(img_batch)
            train_mask_batches.append(mask_batch)
            img_batch = []
            mask_batch = []
            counter = 0
        else:
            img_batch.append(img)
            mask_batch.append(mask)
            counter += 1
    train_img_batches.append(img_batch)
    train_mask_batches.append(mask_batch) 
    return train_img_batches, train_mask_batches

In [92]:
def batch_reader(batch, mask = 0):
    res = []
    for path in batch:
        img = tiff.imread(path)
        res.append(img/255)
    if mask:
        return np.stack([res], axis=-1)
    else:
        return np.stack(res)

In [95]:
def batch_training(train_img_batches, train_mask_batches, model, eps):
    for img_batch, mask_batch in zip(train_img_batches, train_mask_batches):
        input_tensor = batch_reader(img_batch)
        output_tensor = batch_reader(mask_batch,mask=1)
        # print(input_tensor.shape,' ### ',output_tensor.shape)
        model.fit(x=input_tensor, y=output_tensor, epochs = eps, validation_split=0.2)
    return model

In [105]:
def model_param_setting(model_save_dir):
    with tf.device('/cpu:0'):
        input_img = Input((256, 256, 3), name='img')
        model = get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)
        model.compile(optimizer=Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics='accuracy')
        callbacks = [keras.callbacks.ModelCheckpoint(model_save_dir+'LCSegModel(FD_Ted).h5', save_best_only=True)]
        return model, callbacks

In [103]:
train_img_batches, train_mask_batches = make_batches(wsi_dir, mask_dir)
model_save_dir = 'D:/LCDC/Dataset/Notebooks/'
callbacks, model = model_param_setting(model_save_dir)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 256, 256, 64) 1792        img[0][0]                        
__________________________________________________________________________________________________
batch_normalization_36 (BatchNo (None, 256, 256, 64) 256         conv2d_38[0][0]                  
__________________________________________________________________________________________________
activation_36 (Activation)      (None, 256, 256, 64) 0           batch_normalization_36[0][0]     
____________________________________________________________________________________________

In [104]:
print(len(train_img_batches),len(train_mask_batches))
# train_img_batches

16730 16730


### Model Hyper parameters and Training

In [18]:
with tf.device('/cpu:0'):
    input_img = Input((256, 256, 3), name='img')
    model = get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)
    model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 1792        img[0][0]                        
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 64) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [19]:
with tf.device('/cpu:0'):
    model.compile(optimizer=Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics='accuracy')
    model_save_dir = 'D:/LCDC/Dataset/Notebooks/'
    callbacks = [keras.callbacks.ModelCheckpoint(model_save_dir+'LCSegModel(FD_Ted).h5', save_best_only=True)]

In [25]:
with tf.device('/device:GPU:0'):
    history = model.fit(x=train_imgs,y = train_masks, epochs=15,callbacks=callbacks)

ValueError: `y` argument is not supported when using `keras.utils.Sequence` as input.